In [ ]:
#Downloading libraries
!pip install -U sentence-transformers

     |████████████████████████████████| 92kB 6.4MB/s 
     |████████████████████████████████| 2.5MB 25.4MB/s 
     |████████████████████████████████| 1.2MB 30.0MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 3.3MB 48.2MB/s 
     |████████████████████████████████| 901kB 57.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=7059e26cd3d50cfddb60a9bbedcd7fa03ccd65e86f90343bfe833e201b3c3479
  Stored in directory: /root/.cache/pip/wheels/38/d2/98/d191289a877a34c68aa67e05179521e060f96394a3e9336be6
Successfully built sentence-transformers
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.14 which is incompatible.


In [ ]:
pip install umap-learn

     |████████████████████████████████| 81kB 5.9MB/s 
     |████████████████████████████████| 1.1MB 18.4MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-cp37-none-any.whl size=76569 sha256=c43399b58b0490f0638d482d17b76b7f45fb74d9f1fe007682e4f72e13a916e9
  Stored in directory: /root/.cache/pip/wheels/ad/df/d5/a3691296ff779f25cd1cf415a3af954b987fb53111e3392cf4
  Created wheel for pynndescent: filename=pynndescent-0.5.4-cp37-none-any.whl size=52374 sha256=72822ee6e2a77f40eec444d3c05e1a4c8234dc9414bc4281cbfacda92641dfaf
  Stored in directory: /root/.cache/pip/wheels/42/4b/8c/f6f119c67cf6583bb192431fa8f7278cf95e5b943055077d94
Successfully built umap-learn pynndescent


In [ ]:
pip install fasttext


     |████████████████████████████████| 71kB 5.3MB/s  eta 0:00:01
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3097198 sha256=4315641bc80f0c25ce20598b79fde9d3906c7d64f4790fe79ba10fe23909fc73
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
#Importing dependencies
import gensim
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim import corpora
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from collections import Counter
from sklearn.metrics import silhouette_score
from wordcloud import WordCloud
from gensim.models.coherencemodel import CoherenceModel
import os
import umap.umap_ as umap
import copy
from datetime import datetime
import warnings
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from wordcloud import WordCloud
import contextlib
import spacy
import re
import random
import math
import fasttext.util

Util

In [ ]:
def get_topic_words(token_lists, labels, k=None):
    """
    get top words within each topic from clustering results
    """
    if k is None:
        k = len(np.unique(labels))
    topics = ['' for _ in range(k)]
    for i, c in enumerate(token_lists):
        topics[labels[i]] += (' ' + ' '.join(c))
    word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
    # get sorted word counts
    word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True), word_counts))
    # get topics
    topics = list(map(lambda x: list(map(lambda x: x[0], x[:10])), word_counts))

    return topics



def get_coherence(model, token_lists, measure='c_v'):
    """
    Get model coherence from gensim.models.coherencemodel
    :param model: Topic_Model object
    :param token_lists: token lists of docs
    :param topics: topics as top words
    :param measure: coherence metrics
    :return: coherence score
    """
    if model.method == 'LDA':
        cm = CoherenceModel(model=model.ldamodel, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
    else:
        topics = get_topic_words(token_lists, model.cluster_model.labels_)
        cm = CoherenceModel(topics=topics, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
    return cm.get_coherence()





def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel



In [ ]:
#word2vec, glove, fasttext
class WordVecVectorizer(object):
    def __init__(self, model, method):
        self.model = model
        self.method = method

        if method == 'WORD2VEC':
          self.dim = 100 
                     
        elif method == 'GLOVE':
           self.dim = 5 
           
        elif method == 'FASTTEXT':           
            self.dim = 300
       
        
    def fit(self, X, y):
        return self    
        
    def transform(self, X):
        return np.array([
            np.mean([self.model[w] for w in texts.split() if w in self.model]
                    or [np.zeros(self.dim)], axis=0)
            for texts in X              
        ])

In [ ]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 
tweet_tokenizer = TweetTokenizer()   
from gensim.models.wrappers import FastText
from gensim.scripts.glove2word2vec import glove2word2vec

def preprocess(docs):
        """
        Preprocess the data
        """        

        print('Preprocessing texts ...')
        sentences = docs['clean_sentences']  # sentence level preprocessed
        token_lists=[] # word level preprocessed
        for sentence in sentences:
          tokens = tweet_tokenizer.tokenize(str(sentence))
          token_lists.append(tokens)

        print('Preprocessing . Done!')
        return sentences, token_lists   

# define model object
class Topic_Model:
    def __init__(self, k=4, method='TFIDF'):
        """
        :param k: number of topics
        :param method: method chosen for the topic model
        """
        if method not in {'TFIDF', 'LDA', 'BERT', 'WORD2VEC', 'GLOVE', 'FASTTEXT'}:
            raise Exception('Invalid method!')
        self.k = k
        self.dictionary = None
        self.corpus = None
        # self.stopwords = None
        self.cluster_model = None
        self.ldamodel = None
        self.vec = {}
        self.gamma = 15  # parameter for reletive importance of lda
        self.method = method
        self.AE = None
        self.id = method + '_' + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")



    def vectorize(self, sentences, token_lists, method=None):
        """
        Get vector representations from selected methods
        """
        # Default method
        if method is None:
            method = self.method

        # turn tokenized documents into a id <-> term dictionary
        self.dictionary = corpora.Dictionary(token_lists)
        # convert tokenized documents into a document-term matrix
        self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        if method == 'TFIDF':
            print('Getting vector representations for TF-IDF ...')
            tfidf = TfidfVectorizer()
            vec = tfidf.fit_transform(sentences)
            print('Getting vector representations for TF-IDF. Done!')
            return vec

        elif method == 'LDA':
            print('Getting vector representations for LDA ...')
            if not self.ldamodel:
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary,
                                                                passes=10, random_state=13)

            def get_vec_lda(model, corpus, k):
                """
                Get the LDA vector representation (probabilistic topic assignments for all documents)
                :return: vec_lda with dimension: (n_doc * n_topic)
                """
                n_doc = len(corpus)
                vec_lda = np.zeros((n_doc, k))
                for i in range(n_doc):
                    # get the distribution for the i-th document in corpus
                    for topic, prob in model.get_document_topics(corpus[i]):
                        vec_lda[i, topic] = prob

                return vec_lda

            vec = get_vec_lda(self.ldamodel, self.corpus, self.k)
            print('Getting vector representations for LDA. Done!')
            return vec

        elif method == 'BERT':

            print('Getting vector representations for BERT ...')
            model = SentenceTransformer('indobenchmark/indobert-base-p1')
            vec = np.array(model.encode(sentences, show_progress_bar=True))
            print('Getting vector representations for BERT. Done!')
            return vec

        elif method == 'WORD2VEC':

            print('Getting vector representations for WORD2VEC ...')
            model = gensim.models.KeyedVectors.load_word2vec_format('drive/My Drive/Thesis/pretrained_model/word2vec/model.bin', binary=True) 
            wtv_vect = WordVecVectorizer(model, method)
            embeddings = wtv_vect.transform(sentences)
            vec = np.vstack(embeddings)
            print('Getting vector representations for WORD2VEC. Done!')
            return vec
        
        elif method == 'GLOVE':

            print('Getting vector representations for GLOVE ...')
            model = loadGloveModel("drive/My Drive/Thesis/Data/gensim_glove_vectors.txt") 
            wtv_vect = WordVecVectorizer(model, method)
            embeddings = wtv_vect.transform(sentences)
            vec = np.vstack(embeddings)
            #print('Getting vector representations for GLOVE. Done!')
            return vec

        elif method == 'FASTTEXT':

            print('Getting vector representations for FASTTEXT ...')
            #loading the model
            model = fasttext.load_model('drive/My Drive/Thesis/pretrained_model/fasttext/model.bin') 
            wtv_vect = WordVecVectorizer(model, method)
            embeddings = wtv_vect.transform(sentences)
            vec = np.vstack(embeddings)
            print('Getting vector representations for FASTTEXT. Done!')
            return vec

                         


    def fit(self, sentences, token_lists, method=None, m_clustering=None):
        """
        Fit the topic model for selected method given the preprocessed data
        :docs: list of documents, each doc is preprocessed as tokens
        :return:
        """
        # Default method
        if method is None:
            method = self.method
        # Default clustering method
        if m_clustering is None:
            m_clustering = KMeans

        # turn tokenized documents into a id <-> term dictionary
        if not self.dictionary:
            self.dictionary = corpora.Dictionary(token_lists)
            # convert tokenized documents into a document-term matrix
            self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        ####################################################
        #### Getting ldamodel or vector representations ####
        ####################################################

        if method == 'LDA':
            if not self.ldamodel:
                print('Fitting LDA ...')
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary,
                                                                passes=20)
                print('Fitting LDA Done!')
        else:
            print('Clustering embeddings ...')
            self.cluster_model = m_clustering(self.k)
            self.vec[method] = self.vectorize(sentences, token_lists, method)
            self.cluster_model.fit(self.vec[method])
            print('Clustering embeddings. Done!')


May

In [ ]:
documents = pd.read_csv("tweet_processed.csv")
documents

,date,tweet,clean_sentences
0,2021-05-01 23:57:34+00:00,Merinding rasanya ngeliat lautan manusia di Ta...,merinding lihat laut manusia tanah abang kmare...
1,2021-05-01 23:57:00+00:00,Walikota Jakarta pusat sebelumnya sudah mengin...,walikota jakarta pusat warga belanja pakai leb...
2,2021-05-01 23:55:47+00:00,sebagai orang yg masih percaya bahwa covid mas...,orang percaya covid hidup orang percaya covid ...
3,2021-05-01 23:54:40+00:00,"Bisa²nya mimpi gua punya istana, trs lg ngada...",mimpi istana ngadain acara didatengin satgas c...
4,2021-05-01 23:54:28+00:00,"Saudara kakekku kena covid, seluarga di isolas...",saudara kakek kena covid seluarga isolasi mandiri
...,...,...,...
43059,2021-05-31 03:08:27+00:00,Para ‘generasi milenial’ diajak untuk terus me...,generasi milenial ajak promosi pariwisata sumu...
43060,2021-05-31 03:08:02+00:00,Apa ertinya lockdown kalau kilang aku still ke...,erti lockdown kilang kena kerja bhahahaha beli...
43061,2021-05-31 03:07:56+00:00,Gue yakin sih biarpun private pasti bakal ada ...,biarpun private shawol jinki privat ramai liha...
43062,2021-05-31 03:07:22+00:00,"Corona.... Corona,,, dirimu merusak nalarku....",corona corona rusak nalar


In [ ]:
import warnings
warnings.filterwarnings('ignore', category=Warning)

def main():
    methodArr = np.array(['LDA', 'WORD2VEC', 'GLOVE']) #Topic Modeling Techniques
    for method in methodArr:
      ntopic = 1
      ntopic_max = 10  
      print("Method : ", method)
      for x in range(ntopic_max):
          print("Number of topic: ", ntopic)
          data = documents 
          data = data.fillna('')  # only the comments has NaN's
          sentences, token_lists = preprocess(data)
          # Define the topic model object
          tm = Topic_Model(k = ntopic, method = method)
          # Fit the topic model by chosen method
          tm.fit(sentences, token_lists)
          # Evaluate using metrics   
          print('Coherence:', get_coherence(tm, token_lists, 'c_v'))
          ntopic = ntopic +1


    
   

In [ ]:
main()

Method :  LDA
Number of topic:  1
Preprocessing texts ...
Preprocessing . Done!
Fitting LDA ...
Fitting LDA Done!
Coherence: 0.3173142144647164
Number of topic:  2
Preprocessing texts ...
Preprocessing . Done!
Fitting LDA ...
Fitting LDA Done!
Coherence: 0.38563725544073646
Number of topic:  3
Preprocessing texts ...
Preprocessing . Done!
Fitting LDA ...
Fitting LDA Done!
Coherence: 0.36641843170260574
Number of topic:  4
Preprocessing texts ...
Preprocessing . Done!
Fitting LDA ...
Fitting LDA Done!
Coherence: 0.44831112361429815
Number of topic:  5
Preprocessing texts ...
Preprocessing . Done!
Fitting LDA ...
Fitting LDA Done!
Coherence: 0.48784489115014473
Number of topic:  6
Preprocessing texts ...
Preprocessing . Done!
Fitting LDA ...
Fitting LDA Done!
Coherence: 0.43451202932402316
Number of topic:  7
Preprocessing texts ...
Preprocessing . Done!
Fitting LDA ...
Fitting LDA Done!
Coherence: 0.5290734002724106
Number of topic:  8
Preprocessing texts ...
Preprocessing . Done!
Fitti

#######################